## `Alexander Kurdyukov BS21-AI-01`

# Context-sensitive Spelling Correction

The goal of the assignment is to implement context-sensitive spelling correction. The input of the code will be a set of text lines and the output will be the same lines with spelling mistakes fixed.

Submit the solution of the assignment to Moodle as a link to your GitHub repository containing this notebook.

Useful links:
- [Norvig's solution](https://norvig.com/spell-correct.html)
- [Norvig's dataset](https://norvig.com/big.txt)
- [Ngrams data](https://www.ngrams.info/download_coca.asp)

Grading:
- 60 points - Implement spelling correction
- 20 points - Justify your decisions
- 20 points - Evaluate on a test set


## Implement context-sensitive spelling correction

Your task is to implement context-sensitive spelling corrector using N-gram language model. The idea is to compute conditional probabilities of possible correction options. For example, the phrase "dking sport" should be fixed as "doing sport" not "dying sport", while "dking species" -- as "dying species".

The best way to start is to analyze [Norvig's solution](https://norvig.com/spell-correct.html) and [N-gram Language Models](https://web.stanford.edu/~jurafsky/slp3/3.pdf).

You may also want to implement:
- spell-checking for a concrete language - Russian, Tatar, etc. - any one you know, such that the solution accounts for language specifics,
- some recent (or not very recent) paper on this topic,
- solution which takes into account keyboard layout and associated misspellings,
- efficiency improvement to make the solution faster,
- any other idea of yours to improve the Norvig’s solution.

IMPORTANT:  
Your project should not be a mere code copy-paste from somewhere. You must provide:
- Your implementation
- Analysis of why the implemented approach is suggested
- Improvements of the original approach that you have chosen to implement

### Importing all necessary packages
In this implementation it was decided to use already codded spell corrector due to the fact that otherwise it was needed to write(indeed copy already exisiting) solution and in order to reduce amount of code to check(which anyways would have been just copied) and to not reinvent a wheel the corrector with the same edit1 and edit2 functions implemented will be used as python package. 

In [6]:
from autocorrect import Speller
import pandas as pd
from tqdm import tqdm
import numpy as np
import re
from collections import Counter

spell = Speller()

### Preparing dictionary with bigram counts, counting total number of bigrams, and saving second grams from pairs
The process is just loading already preprocessed data

In [3]:
bigr_counts = {}            ## Dictionary with counts for each starting gram its complementary second one
sec_bigr_vocab = set()      ## Set of all grams from that are complementary ones 
total_count = 0

## I used 2 prvided datasets(for test and demonstrational purposes I hope it is OK to have only them)
for dataset_path in ["bigrams.txt", "coca_all_links.txt"]:
    df = pd.read_csv(dataset_path, sep='\t', names=["counts", "first_gram", "second_gram"], usecols=[0, 1, 2], encoding='latin-1')
    df[["first_gram", "second_gram"]] = df[["first_gram", "second_gram"]].astype(str)
    total_count += np.sum(df['counts'].astype(np.int64).values)
    bar = tqdm(df.iterrows(), total=df.shape[0], desc=f"Loading {dataset_path}")
    for index, row in bar:
        count, f_gram, s_gram  = row["counts"], row["first_gram"].lower(), row["second_gram"].lower()
        if(f_gram not in bigr_counts):
            bigr_counts[f_gram] = {s_gram: count}
        elif(s_gram not in bigr_counts[f_gram]):
            bigr_counts[f_gram][s_gram] = count
        else:
            bigr_counts[f_gram][s_gram] += count

        sec_bigr_vocab.add(s_gram)

Loading coca_all_links.txt: 100%|██████████| 156306/156306 [00:11<00:00, 13158.84it/s]


### Next, there is code for context sensitve spell correction.  
The implementation was almost not inspired by any open source code in the Internet and hence is in the "testing" and "working in theory" states. It may not be optimised(both in space and exec. time) well and probably performs poor(but it was tried to implement it not to be so).

In [109]:
def calc_prob(cur_gram: str, pred_gram: str, found_probs: dict, depth: int, cur_depth: int = 0):
    """
        Function for calculating prbabilities to construct N-grams from bigrams of curtain N(depth/width)
    """
    if((cur_gram, pred_gram, depth) in found_probs):            ## If this chain was already computed earlier 
        return found_probs[(cur_gram, pred_gram, depth)]        ## then we return the stored result

    if(cur_gram in bigr_counts):                                        ## If the current gram starts any known bigram
        all_count = sum(bigr_counts[cur_gram].values())                 ## then we sum up all the counts of bigrams that can be constructed and are known
        if(cur_depth + 1 == depth):                                     ## If the needed depth is reached 
            gr_count = bigr_counts[cur_gram].get(pred_gram, -1.0)       ## then we get counts of bigram that is constructed from the current gram and the gram that is supposed to end the N-gram
            if(gr_count != -1):                                         ## If their are such known bigrams 
                to_sum = gr_count / all_count                           ## then the probability of constructing it is following: number of needed bigrams / number of all bigrams both of which can be constructed from the current gram
            else:
                to_sum = 1 / (all_count + len(bigr_counts[cur_gram]))   ## Otherwise, we suppose that there we addeded len(bigr_counts[cur_gram]) new bigrams to the dictionary and 1 of them is the needed one
            return to_sum                                               ## And after that we return the probability
        else:
            prob_sum = 0.0                                              ## If the needed depth was not reached 
            for gram in bigr_counts[cur_gram].items():                  ## Then we iterate through all the grams that are complementary to the current one 
                prob_sum += (gram[1] / all_count * calc_prob(gram[0], pred_gram, found_probs, depth, cur_depth+1))   ## And calculate the probability as: probability to pick the gram as complementary * probability to pick the next complpementary gram
            found_probs[(cur_gram, pred_gram, depth)] = prob_sum                                                     ## In such a way we calculate the probability of constructing an N-gram that starts and ends with needed grams
            # print(prob_sum, cur_gram, pred_gram)
            return prob_sum     ## Do not forget to return probabilities in all cases
    return 1 / total_count      ## If from the chain breaks from the very start - return some small value

def fix(text: list, beam_width: int = 3, wind_size: int = 3):
    """
        Function that fixes the given text with the use of bigram and word frequencies 
    """
    found_probs = {}    ## Dictionary for memorizing already checked grams chains with curtain depth/width
    fixed = [[]]    ## List of all generated versions/fixes of the passed text
    poses = []      ## Positions of the words that are classified by the algorithm as "words with typos"
    probs = [0.0]   ## Log10(P)'s of the versions/fixes - without "log10" just probabilities to fix the text in a curtain way
    
    ## Iterating through all the words/tokens/grams in the text
    for id, word in enumerate(text):
        ## Getting candidates for each word/token/gram and sorting them by frequency
        candidates = sorted([cand for cand in spell.get_candidates(word) if (cand[1] in bigr_counts or cand[1] in sec_bigr_vocab)], key=lambda pair: pair[0], reverse=True)
        
        ## If word is "known" then we do not need to change/fix it
        if(word in bigr_counts or word in sec_bigr_vocab or len(candidates) == 0):
            fixed = [fix+[word] for fix in fixed]
        else: 
            candidates = [cand for cand in candidates[:beam_width]]                         ## Otherwise, we take top-beam_width most frequent words
            # print(candidates)
            new_fixes = []                                                                  ## Prepare spaces for the new versions
            new_probs = []                                                                  ## and probabilities
            for f_id, fix in enumerate(fixed):                                              ## Iterating over all versions
                for cand in candidates:                                                     ## and adding to each version/fix each candidate word
                    new_fixes.append(fix+[cand[1]])                                         ## Save each version/fix
                    new_probs.append(probs[f_id] + np.log10(cand[0]/candidates[0][0]))      ## and calculate weighted and smoothed probability(idea to log10 probabilities is taken from https://www.kaggle.com/code/dhruvdeshmukh/spelling-corrector-using-n-gram-language-model)
            fixed = new_fixes                                                               ## Replace old versions/fixes with new ones
            probs = new_probs                                                               ## the same with probababilities
            poses.append(id)                                                                ## and add the position of the words we change/fix
    
    ## After all the versions are generated we can try to calculate which version is most "appropriate" according to the context of the original text
    half_size = (wind_size - 1) // 2 + 1                                                                ## Firstly, we calculate max size of the N-grams which we will use in order to try to consider context of the oroginal text(all the wind_size's are odd numbers are odd and > 1... just for simplicity)
    for f_id, fix in enumerate(fixed):                                                                  ## Iterate over all the versions/fixes
        for pos in poses:                                                                               ## Iterate over all the positions of words to be changed/fixed 
            for neigh_diff in range(1, half_size):                                                      ## Iterate over all the Ks for K-grams constructions
                if(pos - neigh_diff > -1):                                                              ## If we not overcome the start of the text
                    probs[f_id] += np.log10(calc_prob(fix[pos - neigh_diff], fix[pos], found_probs, neigh_diff))    ## Then we compute smoothed probability of the fixed word to be at the end of gram with lengh = neigh_diff
                if(pos + neigh_diff < len(fix)):                                                                    ## If we not overcome the end of the text
                    probs[f_id] += np.log10(calc_prob(fix[pos], fix[pos + neigh_diff], found_probs, neigh_diff))    ## Then we compute smoothed probability of the fixed word to be at the beginning of gram with lengh = neigh_diff
    
    """ Some debugging info"""
    # print(fixed)
    # print(probs)
    # print(poses)
    return fixed[np.argmax(probs)]  ## At the very end we return the most probale variant of the changed/fixed text accordind to the algorithm's logic

test = "we wll be dking sport and yow need to abandon their dking man"
result = fix(test.split(), 3, 5)
print(result)

['we', 'will', 'be', 'doing', 'sport', 'and', 'you', 'need', 'to', 'abandon', 'their', 'dying', 'man']


## Justify your decisions

Write down justificaitons for your implementation choices. For example, these choices could be:
- Which ngram dataset to use
- Which weights to assign for edit1, edit2 or absent words probabilities
- Beam search parameters
- etc.

`Due to 2 reasons 2 bigram datasets were used:` 
1) Whole solution was designed to be a test of theory and hence there was no need in large datasets since in such case it was much harder to debug the process. 
2) Among all provided data sets 2 with bigrams were rather large to then use quite large variety of words for test sentences creation.
3) In theory all N-grams(with N > 2) can be divided on bigrams with additional computational steps

`If the word/gram was in the beginning or middle of a N-gram then its probability is 1 / sum of all counts of bigrams(can be changed to number of known distinc words). If the word/gram is in the end of a N-gram then its probability is 1 / sum of counts of known bigrams that can be constructed from the preceding gram.`

`Empirically it was found that values 3 and 4 for a width of a beam are rather effective in a process of spell correction.`

`Window size(or N) for N-grams that are used in considering context are odd numbers > 1: 3, 5 etc. due to simplicity and symmetry property`

#### `All additional justifications can be found as comments to the code or to the headers of cells`

## Evaluate on a test set

Your task is to generate a test set and evaluate your work. You may vary the noise probability to generate different datasets with varying compexity. Compare your solution to the Norvig's corrector, and report the accuracies.

### Next cell is just a copy of [Norvig's solution](https://norvig.com/spell-correct.html)

In [103]:
def words(text): return re.findall(r'\w+', text.lower())

WORDS = Counter(words(open('big.txt').read()))

def P(word, N=sum(WORDS.values())): 
    "Probability of `word`."
    return WORDS[word] / N

def correction(word): 
    "Most probable spelling correction for word."
    return max(candidates(word), key=P)

def candidates(word): 
    "Generate possible spelling corrections for word."
    return (known([word]) or known(edits1(word)) or known(edits2(word)) or [word])

def known(words): 
    "The subset of `words` that appear in the dictionary of WORDS."
    return set(w for w in words if w in WORDS)

def edits1(word):
    "All edits that are one edit away from `word`."
    letters    = 'abcdefghijklmnopqrstuvwxyz'
    splits     = [(word[:i], word[i:])    for i in range(len(word) + 1)]
    deletes    = [L + R[1:]               for L, R in splits if R]
    transposes = [L + R[1] + R[0] + R[2:] for L, R in splits if len(R)>1]
    replaces   = [L + c + R[1:]           for L, R in splits if R for c in letters]
    inserts    = [L + c + R               for L, R in splits for c in letters]
    return set(deletes + transposes + replaces + inserts)

def edits2(word): 
    "All edits that are two edits away from `word`."
    return (e2 for e1 in edits1(word) for e2 in edits1(e1))

### Since the main task is to create contex-sensitive N-gram spell corrector and not to write "smart" text tokenizer it was decided to write simple regex tokenizer(that partially is Norvig's code) and to preprocess the text by hands(sentence separation).
In addition to that, for the test/demonstration of algorithms there will be no automatically generated errors (like [here](https://www.kaggle.com/code/dhruvdeshmukh/spelling-corrector-using-n-gram-language-model)) and all errors will be "made" by hands. It was found that it is enough to see all the advantages/problems that are already known and expected.

In [105]:
def sep(text: list):
    repl = [re.sub("n't", " not", re.sub("can't", "can not", s.lower())) for s in text]
    return [words(s) for s in repl]

### Original text: https://shakespeare.mit.edu/hamlet/full.html#:~:text=Good%20now%2C%20sit,can%20inform%20me%3F
shakespeare_text = ["Goof now, sit drwn, and tell me, he that lnows",   ## Good/down/knows
        "Why this same cttict and most obdervant watch",                ## strict/observant
        "So nightly toils the dibject of the land",                     ## subject
        "And wgy such daily cast of brazen vannin",                     ## why/cannon
        "And foreign mart for onplements of war",                       ## implements
        "Wgy such impress of shipwrights, egose sore task",             ## why/whose
        "Does not duvude the Sinsay from the week",                     ## divide/Sunday
        "What mofht be toward, rhat this sweaty haste",                 ## might/that
        "Doth make the nifhr joint-labourer with the day",              ## night
        "Who is't that can ongorm me?"]                                 ## inform

### Original text: https://en.wikipedia.org/wiki/Natural_language_processing#:~:text=Natural%20language%20processing%20(,based)%20machine%20learning%20approaches.
scient_text = ["Maturak language processing (NLP) is an interdisciplinary subfield of computer scurnce and linguistics",    ## Natural/science
                "It is primarily comverned with giving computers the ability to dipport and manipulate human language",     ## concerned/support
                "It involves processing natural language darasets, such as text corpira or speech corpora, using either rule-based or probabilistic (i.e. statistical and, modt recently, neural network-based) machine learning approaches"]   ## datasets/corpora/most

### Randomly genrated text by the implementer of this solution
speach_text = ["Isn't it obcius for you that he just dooled both of us?",                               ## obvius/fooled
               "Ok, I understood your poimr, but it is stoll hard for me to believe rhat he made it",   ## point/still/that
               "Believe or not, but the dact is still ghere: we are out of mony now!",                  ## fact/here/money
               "We wll be dking sport and yow need to abandon their dking man"]                         ## doing/you/dying

### Here just change the list of strings to test algorithms in fixing text of different style 
seped_text = sep(speach_text)
print(seped_text)

for sent in seped_text:
    result_f = fix(sent, 3, 5)
    result_N = [correction(word) for word in sent]
    print("Context. algo: ", result_f)
    print("Norvig's algo: ", result_N, end="\n\n")

[['is', 'not', 'it', 'obcius', 'for', 'you', 'that', 'he', 'just', 'dooled', 'both', 'of', 'us'], ['ok', 'i', 'understood', 'your', 'poimr', 'but', 'it', 'is', 'stoll', 'hard', 'for', 'me', 'to', 'believe', 'rhat', 'he', 'made', 'it'], ['believe', 'or', 'not', 'but', 'the', 'dact', 'is', 'still', 'ghere', 'we', 'are', 'out', 'of', 'mony', 'now'], ['we', 'wll', 'be', 'dking', 'sport', 'and', 'yow', 'need', 'to', 'abandon', 'their', 'dking', 'man']]
Context. algo:  ['is', 'not', 'it', 'obvious', 'for', 'you', 'that', 'he', 'just', 'cooled', 'both', 'of', 'us']
Norvig's algo:  ['is', 'not', 'it', 'obvious', 'for', 'you', 'that', 'he', 'just', 'doomed', 'both', 'of', 'us']

Context. algo:  ['ok', 'i', 'understood', 'your', 'point', 'but', 'it', 'is', 'stoll', 'hard', 'for', 'me', 'to', 'believe', 'that', 'he', 'made', 'it']
Norvig's algo:  ['ok', 'i', 'understood', 'your', 'power', 'but', 'it', 'is', 'still', 'hard', 'for', 'me', 'to', 'believe', 'that', 'he', 'made', 'it']

Context. algo:

## Conclusion:
    There are 3 types of situations during the process of correction:
1) The written algorithm corrects the words that are not supposed to be corrected. `Reason: small "knowledge" dataset`
2) The written algorithm corrects the words in a wrong way(out of context/not as in a original text): `Reasons: small/unbalanced "knowledge" dataset or lack of regularizing coefficients/additional constraints to work with naturally unbalanced word frequencies. Example: "strict"->"attics" even with the fact that probs of choosing "strict" in most of the cases were much higher`
3) The written algorithm corrects the words in the way as it is expected.

`In comparison with Norvig's solution:`
1) In some cases Norvig's solution performs better. `Reason: sum of reasons written above. Examples: "Good"->"Goof", "toils"->"tools"`
2) In some cases Norvig's solution gives incorrect "fixes" when the written algorithm gives correct ones. `Examples: "whose"->"those", "might"->"most"`
3) In some cases both Norvig's solution and the written algorithm give incorrect "fixes". `Examples: "nlp"->"nl"/"nap", "datasets"->"darasets", "money"->"many"`

All in all, written algorithm can outperform Norvig's solution if to provide larger "knowledge" dataset and improve the code to work better with naturally unbalanced data.\
For now this solution `performs in most cases not worse` than Norvig's algorithm(but if consider the fact that Norvig's solution does not take into account a context of texts...).